In [1]:
import pickle

import altair as alt
import altair_saver

import pandas as pd

import polyclonal

import warnings
warnings.filterwarnings('ignore')

import yaml

from IPython.utils import io

In [2]:
import os
os.chdir('../../')

In [3]:
antibody='197C'
pickle_file=f'results/antibody_escape/{antibody}.pickle'

with open(pickle_file, "rb") as f:
    model = pickle.load(f)

f.close()

In [4]:
with open('data/polyclonal_config.yaml') as f:
    antibody_config = yaml.safe_load(f)[antibody]

In [5]:
muteffects_csv = "results/muteffects_functional/muteffects_observed.csv"
site_numbering_map = "data/site_map.csv"

site_map = pd.read_csv(site_numbering_map).rename(columns={"reference_site": "site"})

df_to_merge = [site_map]

plot_kwargs = antibody_config["plot_kwargs"]

muteffects = pd.read_csv(muteffects_csv).rename(
    columns={"reference_site": "site", "effect": "functional effect"}
)[["site", "mutant", "functional effect"]]
plot_kwargs["addtl_slider_stats"]["functional effect"] = -1.38

plot_kwargs["addtl_slider_stats_hide_not_filter"] = []

plot_kwargs["addtl_slider_stats_hide_not_filter"].append("functional effect")

plot_kwargs["init_floor_at_zero"] = False

plot_kwargs["alphabet"] = ['R', 'K', 'H', 'D', 'E', 'Q', 'N', 'S', 'T', 'Y', 
                           'W', 'F', 'A', 'I', 'L', 'M', 'V', 'G', 'P', 'C']

# plot_kwargs["show_lineplot"] = False

df_to_merge.append(muteffects)

In [6]:
def expand_ranges(range_list):
    expanded_list = []
    for item in range_list:
        if isinstance(item, int):
            expanded_list.append(item)
        elif isinstance(item, str):
            if ':' in item:
                start, end = item.split(':')
                expanded_list.extend(range(int(start), int(end) + 1))
            else:
                expanded_list.append(int(item))
    return expanded_list

# Example usage
ranges = ['122:130', 135, '137:140', '145:170']
expanded_numbers = expand_ranges(ranges)

In [19]:
# sites = ['121:124', 138, 145, '158:160', '188:193']
# sites_to_filter = expand_ranges(sites)

df = model.mut_escape_df

sites_str = [str(num) for num in sites_to_filter]

# Filter the DataFrame
filtered_df = df[(df['site'].isin(sites_str)) & (df['times_seen'] >= 3)]

max_escape = abs(filtered_df['escape_mean']).max()
# max_escape = 1
max_escape

1.019409134129043

In [7]:
max_escape = 1

In [8]:
site_ranges = ['121:124', 138, 145, '158:160', '188:193']

heatmaps = []

for entry in site_ranges:
    sites = expand_ranges([entry])
    heatmap = model.mut_escape_plot(
        df_to_merge=df_to_merge,
        **plot_kwargs,
        show_lineplot=False,
        sites_to_show={"include": sites},
        heatmap_max_fixed=max_escape,
        heatmap_min_fixed=-max_escape
    )

    heatmaps.append(heatmap)

In [9]:
i=0
for entry in heatmaps:
    range = site_ranges[i]
    entry.save(f'scratch_notebooks/figure_drafts/models/{antibody}_{range}_heatmap.png', scale_factor=2.0)
    i+=1

In [15]:
lineplot = model.mut_escape_plot(
    df_to_merge=df_to_merge,
    **plot_kwargs,
    show_heatmap=False
)

lineplot.save(f'scratch_notebooks/figure_drafts/models/{antibody}_lineplot.svg', 
              scale_factor=2.0,
              # engine="altair_saver"
             )

lineplot

ValueError: No enabled saver found that supports format='pdf'

In [17]:
lineplot.save(f'scratch_notebooks/figure_drafts/models/{antibody}_lineplot.svg', 
              scale_factor=2.0,
              # engine="altair_saver" 
             )

In [11]:
model.mut_escape_plot(
    df_to_merge=df_to_merge,
    **plot_kwargs,
    # show_heatmap=False
    sites_to_show={"include": sites_to_filter},
    heatmap_max_fixed=max_escape,
    heatmap_min_fixed=-max_escape
)

alt.VConcatChart(...)